# Dataset information
ref: http://bbci.de/competition/iii/desc_V.html

## Experiment

This dataset contains data from 3 normal subjects during 4 non-feedback sessions. The subjects sat in a normal chair, relaxed arms resting on their legs. There are 3 tasks:
1. Imagination of repetitive self-paced left hand movements, (_left_, class 2),
2. Imagination of repetitive self-paced right hand movements, (_right_, class 3),
3. Generation of words beginning with the same random letter, (_word_, class 7).

All 4 sessions of a given subject were acquired on the same day, each lasting 4 minutes with 5-10 minutes breaks in between them. The subject performed a given task for about 15 seconds and then switched randomly to another task at the operator's request. EEG data is not splitted in trials since the subjects are continuously performing any of the mental tasks. The algorithm should provide an output every 0.5 seconds using the last second of data (see clarification in the paragraph 'Requirements and Evaluation'.) Data are provided in two ways:
1. _Raw EEG_ signals. Sampling rate was 512 Hz.
2. _Precomputed_ features. The raw EEG potentials were first spatially filtered by means of a surface Laplacian. Then, every 62.5 ms --i.e., 16 times per second-- the power spectral density (PSD) in the band 8-30 Hz was estimated over the last second of data with a frequency resolution of 2 Hz for the 8 centro-parietal channels C3, Cz, C4, CP1, CP2, P3, Pz, and P4. As a result, an EEG sample is a 96-dimensional vector (8 channels times 12 frequency components).

## Format of the Data

For each subject there are 3 training files and 1 testing file (the last recording session). Training files are labelled while testing files are not. Data are provided in ASCII format.
- _Precomputed features_: files contain a PSD sample per row (i.e., the first 12 components are the PSD in the band 8-30 Hz at channel C3, and so on, for a total of 96 components). The number of PSD samples are:

| &nbsp; | **Training** | **Testing** |
| --- | :---: | :---: |
| Subject 1 | 3488/3472/3568 | 3504 |
| Subject 2 | 3472/3456/3472 | 3472 |
| Subject 3 | 3424/3424/3440 | 3488 |

- _Raw EEG signals_: each line of the files contains the 32 EEG potentials acquired at a given time instant in the order: Fp1, AF3, F7, F3, FC1, FC5, T7, C3, CP1, CP5, P7, P3, Pz, PO3, O1, Oz, O2, PO4, P4, P8, CP6, CP2, C4, T8, FC6, FC2, F4, F8, AF4, Fp2, Fz, Cz. In the training files, each line has a 33rd component indicating the class label.

## Requirements and Evaluation

Please provide your estimated class labels (2, 3, or 7) for every input vector of the 3 test files (one per subject). The labels must be estimated in the following way:

- **Precomputed features:** Since input vectors are computed 16 times per second, provide the average of 8 consecutive samples (so that to get a response every 0.5 seconds). Other (i.e. also past) samples must not be used in order to guarantee a fast response times of the system, although for the competition test data set averaging over more samples could be of benefit.

- **Raw signals:** Compute vectors 16 times per second using the last second of data. Then provide the average of 8 consecutive samples (so that to get a response every 0.5 seconds). Other (i.e. also past) samples must not be used in order to guarantee a fast response times of the system, although for the competition test data set averaging over more samples could be of benefit.

Also give a description of the used algorithm. The performance measure is the classification accuracy (correct classification divided by the total number of samples) averaged over the 3 subjects. 
There will be a special prize to the best algorithm working with the precomputed samples (in the case it does not achieve the best absolute result).

## Technical Information

EEG signals were recorded with a Biosemi system using a cap with 32 integrated electrodes located at standard positions of the International 10-20 system. The sampling rate was 512 Hz. Signals were acquired at full DC. No artifact rejection or correction was employed.

Quick summary:

*   Continuous, multi-class dataset.
*   8 channels (electrode positions) * 12 frequencies (8:2:30 Hz) = 96 dimensional vector
*   Small data size

---
# Main Tasks

In [0]:
import os
from scipy.io import loadmat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
%matplotlib inline

import pickle
from collections import Counter
import re

from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

## Read Data

### Mount google drive - Skip this if running locally

In [0]:
# Mount Google drive
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/gdrive')

In [0]:
# Here you must try by yourself where the path to the dataset folder is !!
# e.g. !ls "gdrive/My Drive/Colab Notebooks/DME/datasets"
!ls "gdrive"

In [0]:
# Then assign the path here.
# This is the path for my GG drive, change it according to your path
# dataset_path = "gdrive/My Drive/Colab Notebooks/DME/datasets"

# Your folder path here
dataset_path = "gdrive/My Drive/Colab Notebooks/DME/datasets"

### Read data from csv

In [0]:
try:
  dataset_path
except:
  dataset_path = os.path.join(os.getcwd(), 'datasets')

df_train_pcf = pd.read_csv(os.path.join(dataset_path, 'df_train_pcf.csv'))
df_test_pcf = pd.read_csv(os.path.join(dataset_path, 'df_test_pcf.csv'))
df_train_raw = pd.read_csv(os.path.join(dataset_path, 'df_train_raw.csv'))
df_test_raw = pd.read_csv(os.path.join(dataset_path, 'df_test_raw.csv'))

label_dict = {2: 'left', 3: 'right', 7: 'word'}
df_train_pcf.replace({'label' : label_dict}, inplace=True)
df_train_raw.replace({'label' : label_dict}, inplace=True)

pcf_setup = pickle.load(open(os.path.join(dataset_path, 'pcf_setup.obj'), "rb"))
raw_setup = pickle.load(open(os.path.join(dataset_path, 'raw_setup.obj'), "rb"))

pcf_channels = pcf_setup['channels']
raw_channels = raw_setup['channels']

### !!! Skip this step !!! if no needs of MAT files !!!
### Read data from seperated mat files

In [0]:
def load_datasets(filenames, folder=os.path.join(os.getcwd(), 'datasets'), mapping_fn=None, test=False):
    list_df = []
    for file in filenames:
        path = os.path.join(folder, file)
        mat = loadmat(path)
        list_df.append(mat_to_df(mat, mapping_fn=mapping_fn, ignore_label=test))
    return pd.concat(list_df, ignore_index=True)

def psd_mapping(mat):
    # Return X, Y, filename, channels, n_components
    return {
        'X': mat.get('X'),
        'Y': mat.get('Y'),
        'filename': mat['nfo']['name'][0][0][0],
        'channels': [c for [c] in mat['nfo']['clab'][0][0][0]],
        'n_components': 12,
        'fs': mat['nfo']['fs'][0][0][0][0],
        'xpos': mat['nfo']['xpos'][0][0].reshape(-1),
        'ypos': mat['nfo']['ypos'][0][0].reshape(-1)
    }

def raw_eeg_mapping(mat):
    # Return X, Y, filename, channels, n_components
    return {
        'X': mat.get('X'),
        'Y': mat.get('Y'),
        'filename': mat['nfo']['name'][0][0][0],
        'channels': [c for [c] in mat['nfo']['clab'][0][0][0]],
        'n_components': 1,
        'fs': mat['nfo']['fs'][0][0][0][0],
        'xpos': mat['nfo']['xpos'][0][0].reshape(-1),
        'ypos': mat['nfo']['ypos'][0][0].reshape(-1)
    }

def mat_to_df(mat, mapping_fn=None, ignore_label=False):

    assert callable(mapping_fn), "Missing mapping function (mapping_fn).\nInput: mat\nOutput: X, Y, filename, channels, n_components"

    data = mapping_fn(mat)
    X = data['X']
    Y = data['Y']
    filename = data['filename']
    channels = data['channels']
    n_components = data['n_components']

    pattern = '^(?:train|test)_subject(\d+)_\w+?(\d+)$'
    match = re.search(pattern, filename)

    assert match is not None, "Filename does not match regex '{}'".format(pattern)

    subject = int(match.group(1))
    session = int(match.group(2))

    n_rows, n_cols = X.shape

    if n_components > 1:
        columns = ["{}_{}".format(c,i+1) for c in channels for i in range(n_components) ] + ['subject', 'session']
    else:
        columns = channels + ['subject', 'session']
    dtypes = [np.float64] * n_cols + [np.uint8] * 3
    mixed_data = np.hstack((
        X,
        subject * np.ones((n_rows, 1), dtype=int),
        session * np.ones((n_rows, 1), dtype=int),
    ))

    if ignore_label == False:
        assert Y is not None, "Missing label data.\nSolution: mat_to_df(mat, ignore_label=True)"
        columns += ['label']
        dtypes += [np.uint8]
        mixed_data = np.hstack((mixed_data, Y))

    df = pd.DataFrame(mixed_data, columns=columns)
    df = df.astype(dict(zip(columns, dtypes)))
    return df

def extract_channels(filename, folder=os.path.join(os.getcwd(), 'datasets'), mapping_fn=None):
    path = os.path.join(folder, filename)
    mat = loadmat(path)
    data = mapping_fn(mat)
    return data['channels']

def extract_setup(filename, folder=os.path.join(os.getcwd(), 'datasets'), mapping_fn=None):
    path = os.path.join(folder, filename)
    mat = loadmat(path)
    setup = ['channels', 'fs', 'xpos', 'ypos']
    return {k: v for k, v in mapping_fn(mat).items() if k in setup}


In [0]:
train_precomputed_features_data_files = [
    'train_subject1_psd01.mat',
    'train_subject1_psd02.mat',
    'train_subject1_psd03.mat',
    'train_subject2_psd01.mat',
    'train_subject2_psd02.mat',
    'train_subject2_psd03.mat',
    'train_subject3_psd01.mat',
    'train_subject3_psd02.mat',
    'train_subject3_psd03.mat'
]
test_precomputed_features_data_files = [
    'test_subject1_psd04.mat',
    'test_subject2_psd04.mat',
    'test_subject3_psd04.mat'
]
train_raw_eeg_signals_data_files = [
    'train_subject1_raw01.mat',
    'train_subject1_raw02.mat',
    'train_subject1_raw03.mat',
    'train_subject2_raw01.mat',
    'train_subject2_raw02.mat',
    'train_subject2_raw03.mat',
    'train_subject3_raw01.mat',
    'train_subject3_raw02.mat',
    'train_subject3_raw03.mat'
]
test_raw_eeg_signals_data_files = [
    'test_subject1_raw04.mat',
    'test_subject2_raw04.mat',
    'test_subject3_raw04.mat'
]

df_train_pcf = load_datasets(train_precomputed_features_data_files, mapping_fn=psd_mapping)
df_test_pcf = load_datasets(test_precomputed_features_data_files, mapping_fn=psd_mapping, test=True)
df_train_raw = load_datasets(train_raw_eeg_signals_data_files, mapping_fn=raw_eeg_mapping)
df_test_raw = load_datasets(test_raw_eeg_signals_data_files, mapping_fn=raw_eeg_mapping, test=True)

pcf_setup = extract_setup(train_precomputed_features_data_files[0], mapping_fn=psd_mapping)
raw_setup = extract_setup(train_raw_eeg_signals_data_files[0], mapping_fn=raw_eeg_mapping)

In [0]:
# Save data as csv
df_train_pcf.to_csv(os.path.join(os.getcwd(), 'datasets', 'df_train_pcf.csv'), index=False)
df_test_pcf.to_csv(os.path.join(os.getcwd(), 'datasets', 'df_test_pcf.csv'), index=False)
df_train_raw.to_csv(os.path.join(os.getcwd(), 'datasets', 'df_train_raw.csv'), index=False)
df_test_raw.to_csv(os.path.join(os.getcwd(), 'datasets', 'df_test_raw.csv'), index=False)

import pickle
with open(os.path.join(os.getcwd(), 'datasets', 'pcf_setup.obj'), "wb") as f:
    pickle.dump(pcf_setup, f, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(os.getcwd(), 'datasets', 'raw_setup.obj'), "wb") as f:
    pickle.dump(raw_setup, f, pickle.HIGHEST_PROTOCOL)

## DBDA
ref: https://publications.idiap.ch/downloads/papers/2008/Galan_THESIS_2008.pdf

In [0]:
def df_channel_norm(df, channels):
    # output: [0,1]
    data = df.copy()
    for channel in pcf_channels:
        df_channel = data.filter(regex=channel)
        data[df_channel.columns] = df_channel.div(df_channel.sum(axis=1), axis='index')
    return data

def df_channel_standardize(df, channels):
    # output: zero mean, unit variance
    data = df.copy()
    for channel in pcf_channels:
        df_channel = data.filter(regex=channel)
        data[df_channel.columns] = df_channel.sub(df_channel.mean(axis=1), axis='index').div(df_channel.std(axis=1), axis='index')
    return data

def cvt(df):
    data = df.copy()
    data_columns = data.drop(columns=['label', 'session', 'subject']).columns

    n = len(data)
    n_l = data['label'].value_counts()
    labels = n_l.keys()

    m = data[data_columns].mean()
    m_l = { label: data[data['label'] == label][data_columns].mean() for label in labels }

    d = len(data_columns) # number of feature
    B = np.zeros((d, d))
    W = np.zeros((d, d))
    for label in labels:
        diff = np.array(m_l[label] - m).reshape((d, 1))
        B += n_l[label] * diff @ diff.T
        for i, psd in data[data_columns].iterrows():
            diff = np.array(psd - m_l[label]).reshape((d, 1))
            W += diff @ diff.T

    E, A = np.linalg.eigh(np.linalg.inv(W) @ B)
    eigen_vec = A[:, E > 0]
    Y = np.array(data[data_columns]) @ eigen_vec
    return Y, eigen_vec

def proximity_test(X, y, X_test, sampling_rate=8, dsfn=lambda x,y: sum((x-y)**2)):
    """
    Return: list of label prediction according to X_test with sampling rate
    """

    assert sampling_rate > 0, "Sampling rate must be greater than 0 !!"

    N_av = sampling_rate

    labels = np.unique(y)
    N_l = {label: sum(y==label) for label in labels}
    pred = []

    score_geo = {}
    for label in labels:
        score = 0.
        for xli in X[y == label]:
            for xlj in X[y == label]:
                score += dsfn(xli, xlj)
        score /= 2 * N_l[label] * N_l[label]
        score_geo[label] = score

    for t in range(len(X_test) // sampling_rate):
        best = (None, None) # label, score
        for label in labels:
            score = 0.
            for xt in X_test[t*sampling_rate:(t+1)*sampling_rate]:
                score_rel = 0.
                for xlj in X[y == label]:
                    score_rel += dsfn(xt, xlj)
                score_rel /= N_l[label]
                score += score_rel - score_geo[label]
            score /= N_av
            if best[1] is None or best[1] > score:
                best = (label, score)
        pred.append(best)

    return np.array([ t[0] for t in pred ])

def compute_interest_df_index(df, train_input_index, test_input_index):

    train_index = pd.Series([False] * df.shape[0])
    test_index = pd.Series([False] * df.shape[0])

    for subject, session in train_input_index:
        train_index |= (df['subject'] == subject) & (df['session'] == session)
    
    for subject, session in test_input_index:
        test_index |= (df['subject'] == subject) & (df['session'] == session)
        
    all_index = train_index | test_index
    
    return (train_index, test_index, all_index)

def dbda(df, train_input_index, test_input_index, pcf_channels, preprocess='cvt', sampling_rate=8):
    """
    train_input_index: [(subject id, session id), ...]
    test_input_index: [(subject id, session id), ...]
    """

    data = df.copy()
    data_columns = data.drop(columns=['label', 'session', 'subject']).columns
    
    train_index, test_index, all_index = compute_interest_df_index(df, train_input_index, test_input_index)
    
    if preprocess == 'cvt':
        # Normalize
        # data: dataframe
        data = df_channel_norm(data, pcf_channels)

        # CVT
        # Y, eigen_vec: numpy.arrray
        Y, eigen_vec = cvt(data[train_index])

        # Construct variable
        # All variables below are numpy.array
        X_train = Y

        X_val = np.array(data[test_index][data_columns]) @ eigen_vec

    elif preprocess == 'pca':
        # Normalize
        # data: dataframe
        data = df_channel_standardize(data, pcf_channels)

        pca = PCA(n_components=50)
        pca.fit(data[train_index][data_columns])
        X_train = pca.transform(data[train_index][data_columns])
        X_val = pca.transform(data[test_index][data_columns])

    else:
        raise Exception('Mismatched preprocess: either cvt or pca')

    y_train = data[train_index]['label'].values
    y_val = data[test_index]['label'].values[::-sampling_rate][::-1] # label from last timestep each sample

    # Proxity test (DBDA)
    y_pred = proximity_test(X_train, y_train, X_val, sampling_rate=sampling_rate)

    # Print accuracy
    accuracy = accuracy_score(y_val, y_pred)
    print('Train: {}, Test: {}, Accuracy: {:.2f}'.format(train_input_index, test_input_index, accuracy*100))
    
    return accuracy

In [0]:
test_condition = [
    ([(1,1), (1,2)], [(1,3)]),
    ([(2,1), (2,2)], [(2,3)]),
    ([(3,1), (3,2)], [(3,3)]),
    ([(1,1)], [(1,2)]),
    ([(1,1)], [(1,3)]),
    ([(1,2)], [(1,3)]),
    ([(2,1)], [(2,2)]),
    ([(2,1)], [(2,3)]),
    ([(2,2)], [(2,3)]),
    ([(3,1)], [(3,2)]),
    ([(3,1)], [(3,3)]),
    ([(3,2)], [(3,3)]),
]

results_cvt = []

for train_con, test_con in test_condition:
    %time acc = dbda(df_train_pcf, train_con, test_con, pcf_channels, preprocess='cvt')
    results_cvt.append(acc)

Train: [(1, 1), (1, 2)], Test: [(1, 3)], Accuracy: 73.09
CPU times: user 5min 22s, sys: 510 ms, total: 5min 22s
Wall time: 5min 22s
Train: [(2, 1), (2, 2)], Test: [(2, 3)], Accuracy: 58.99
CPU times: user 5min 7s, sys: 590 ms, total: 5min 7s
Wall time: 5min 7s
Train: [(3, 1), (3, 2)], Test: [(3, 3)], Accuracy: 41.40
CPU times: user 5min 4s, sys: 793 ms, total: 5min 5s
Wall time: 5min 5s
Train: [(1, 1)], Test: [(1, 2)], Accuracy: 70.74
CPU times: user 1min 58s, sys: 265 ms, total: 1min 58s
Wall time: 1min 58s
Train: [(1, 1)], Test: [(1, 3)], Accuracy: 72.42
CPU times: user 2min, sys: 279 ms, total: 2min
Wall time: 2min
Train: [(1, 2)], Test: [(1, 3)], Accuracy: 75.34
CPU times: user 2min 11s, sys: 283 ms, total: 2min 11s
Wall time: 2min 11s
Train: [(2, 1)], Test: [(2, 2)], Accuracy: 51.85
CPU times: user 2min 5s, sys: 296 ms, total: 2min 5s
Wall time: 2min 5s
Train: [(2, 1)], Test: [(2, 3)], Accuracy: 54.84
CPU times: user 2min 6s, sys: 282 ms, total: 2min 6s
Wall time: 2min 6s
Train: [

In [0]:
test_condition = [
    ([(1,1), (1,2)], [(1,3)]),
    ([(2,1), (2,2)], [(2,3)]),
    ([(3,1), (3,2)], [(3,3)]),
    ([(1,1)], [(1,2)]),
    ([(1,1)], [(1,3)]),
    ([(1,2)], [(1,3)]),
    ([(2,1)], [(2,2)]),
    ([(2,1)], [(2,3)]),
    ([(2,2)], [(2,3)]),
    ([(3,1)], [(3,2)]),
    ([(3,1)], [(3,3)]),
    ([(3,2)], [(3,3)]),
]

results_pca = []

for train_con, test_con in test_condition:
    %time acc = dbda(df_train_pcf, train_con, test_con, pcf_channels, preprocess='pca')
    results_pca.append(acc)

Train: [(1, 1), (1, 2)], Test: [(1, 3)], Accuracy: 72.42
CPU times: user 5min 17s, sys: 735 ms, total: 5min 17s
Wall time: 5min 17s
Train: [(2, 1), (2, 2)], Test: [(2, 3)], Accuracy: 58.29
CPU times: user 5min 42s, sys: 866 ms, total: 5min 43s
Wall time: 5min 45s
Train: [(3, 1), (3, 2)], Test: [(3, 3)], Accuracy: 42.79
CPU times: user 5min 7s, sys: 801 ms, total: 5min 8s
Wall time: 5min 9s
Train: [(1, 1)], Test: [(1, 2)], Accuracy: 66.13
CPU times: user 2min 6s, sys: 392 ms, total: 2min 6s
Wall time: 2min 6s
Train: [(1, 1)], Test: [(1, 3)], Accuracy: 69.51
CPU times: user 2min 7s, sys: 525 ms, total: 2min 8s
Wall time: 2min 8s
Train: [(1, 2)], Test: [(1, 3)], Accuracy: 75.11
CPU times: user 2min 6s, sys: 450 ms, total: 2min 7s
Wall time: 2min 6s
Train: [(2, 1)], Test: [(2, 2)], Accuracy: 51.16
CPU times: user 2min 4s, sys: 471 ms, total: 2min 5s
Wall time: 2min 5s
Train: [(2, 1)], Test: [(2, 3)], Accuracy: 54.38
CPU times: user 2min 5s, sys: 554 ms, total: 2min 6s
Wall time: 2min 6s
Tr